<a href="https://colab.research.google.com/github/MMoronto/road-distress-detection/blob/master/road_distress_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# # Launch VSCode server
# !curl -fsSL https://code-server.dev/install.sh | sh
# !nohup code-server --port 9000 --auth none &

# # Tunnel its IP using ngrok
# !pip install pyngrok
# from pyngrok import ngrok
# # ngrok.set_auth_token("get from https://dashboard.ngrok.com/auth/your-authtoken, if you want to pay $10/month for a little bit better service")
# url = ngrok.connect(9000)
# print(url)

##**Basic CNN for pothole detectioin using Fastai**##

In [5]:
from pathlib import Path

from fastai.basic_data import DatasetType
from fastai.callbacks import SaveModelCallback
from fastai.metrics import accuracy
from fastai.train import ClassificationInterpretation
from fastai.vision.data import ImageList
from fastai.vision.learner import cnn_learner
from fastai.vision.models import resnet18
from fastai.vision.transform import ResizeMethod, get_transforms
from fastai.vision import imagenet_stats

import pandas as pd

In [1]:
# Check python version
import sys
sys.version

'3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]'

In [3]:
# Install fastai
!pip3 install fastai

In [12]:
# Install pyTorch
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [13]:
!pip3 install spacy
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 6.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [14]:
from fastai.imports import *

In [16]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

ModuleNotFoundError: ignored

##**Data**##
Create a fastai databunch

Specify:     
* image location
* split dataset into testing and training batches
* specify image lebelling schema
* specify transformation and preproc operations to be applied to the images before passing them on to the model
* batch sizes

In [24]:
path = Path('/road-distress-detection/road-distress-detection/all_data')
il = ImageList.from_csv(path, 'train_ids_labels.csv', folder='all_data', suffix='.JPG')
sd = il.split_by_rand_pct(0.15)
ll = sd.label_from_df()
ll = ll.transform(get_transforms(), size=224)
data = ll.databunch(bs=64).normalize(imagenet_stats)

FileNotFoundError: ignored

Add the test set to the databunch:    

In [ ]:
test = (ImageList.from_csv(path, 'test_ids_only.csv', folder='all_data', suffix='.JPG'))
data.add_test(test)

The databunch looks like the followig:

In [ ]:
data

Inspect the imagelist:    

In [ ]:
il

Inspect a data batch:

In [ ]:
data.show_batch(3)

##**Model**##
We need to define the fatai `Learner`. This object contains all we need to train our model, i.e: the databunch, model, loss function & optimiser. The following line of code achieves this:    

In [ ]:
learn = cnn_learner(data, resnet18, metrics=accuracy)

We are telling it to use a ResNet18 and to report the accuracy after each epoch. The other required information i.e. loss function and optimiser are derived from the data and default settings.

In [ ]:
learn.loss_func

In [ ]:
learn.opt_func

These other things were hapenning in the background when creating the learner. The weights of a ResNet18 trained on ImageNet were loaded, the head of the network was replaced with the appropriate size classification layer and all layers were frozen, expect for the last layer whose weights will not be tuned during training.

##**Train**##
Before training our model, we must determine a good learning rate. In fastai, we achieve this with the following lines of code:

In [ ]:
learn.lr_find()
learn.recorder.plot()